# The Project
- QNN for XOR Problem:

  Classiq has an available dataset for training PQC to imitate the XOR gate, similar to how we trained a U-gate to act as a NOT gate. Design a QNN to solve the XOR    problem. Read more on the dataset here.


## Setting The Scene

When running from Google Colab, we need to install Classiq's SDK and authenticate the remote device:

In [18]:
#!pip install classiq numpy torchvision torch


In [ ]:
# import classiq
# classiq.authenticate()

# Step 1 - Create our `torch.nn.Module`

## Step 1.1 - Create Dataset

Our quantum model will be defined and synthesized as follows:

In [166]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
# XORゲートのデータセット
from torch.utils.data import DataLoader
from classiq.applications.qnn.datasets import (
    DatasetNot,
    state_to_weights,
    state_to_label,
)
from torchvision.transforms import Lambda
NUM_QUBITS = 2

DATASET_NOT = DatasetNot(
    3, transform=Lambda(state_to_weights), target_transform=Lambda(state_to_label)
)

DATALOADER_NOT = DataLoader(DATASET_NOT, batch_size=2, shuffle=True)

for data, label in DATALOADER_NOT:
    print(f"Training the following data: {data}")
    print(f"with the following labels: {label}")

data_loader=DATALOADER_NOT

Training the following data: tensor([[0.0000],
        [3.1416]])
with the following labels: tensor([0., 1.])


## Step 1.2 - Create our parametric quantum program

Our quantum model will be defined and synthesized as follows:

In [167]:
from classiq import *





# 量子プログラムの作成
@qfunc
def encoding(input_0: CReal, input_1: CReal, q: QArray[QBit]) -> None:
    RY(theta=input_0, target=q[0])
    RY(theta=input_1, target=q[1])

@qfunc
def entangle(q: QArray[QBit]) -> None:
    CX(control=q[0], target=q[1])

@qfunc
def mixing(weight_0: CReal, weight_1: CReal, weight_2: CReal, q: QArray[QBit]) -> None:
    RX(theta=weight_0, target=q[0])
    RX(theta=weight_1, target=q[1])
    CRX(theta=weight_2, control=q[0], target=q[1])

@qfunc
def main(input_0: CReal, input_1: CReal, weight_0: CReal, weight_1: CReal, weight_2: CReal, weight_3: CReal, weight_4: CReal, weight_5: CReal, res: Output[QArray[QBit]]) -> None:
    allocate(2, res)
    encoding(input_0, input_1, res)
    entangle(res)
    mixing(weight_0, weight_1, weight_2, res)
    entangle(res)
    mixing(weight_3, weight_4, weight_5, res)

model = create_model(main)
quantum_program = synthesize(model)


#show(quantum_program)



## Step 1.3 - Create the Execution and Post-processing

The following example defines a function that takes in a parametric quantum program plus parameters, executes the program, and returns the result. Notes:

1. The code can be executed on a physical computer or on a simulator. In any case, implement the execution using `execute_qnn`.
2. Post-process the result of the execution to obtain a single number (`float`) and a single dimension `Tensor`.


In [168]:
from classiq.applications.qnn.types import (
    MultipleArguments,
    SavedResult,
    ResultsCollection,
)

from classiq.execution import execute_qnn
from classiq.synthesis import SerializedQuantumProgram

from classiq.applications.qnn.datasets import DATALOADER_NOT


# 実行およびポストプロセス関数の作成
def execute(quantum_program: SerializedQuantumProgram, arguments: MultipleArguments) -> ResultsCollection:
    return execute_qnn(quantum_program, arguments)

def post_process(result: SavedResult) -> torch.Tensor:
    counts = result.value.counts
    p_zero = float(counts.get("0", 0.0) / sum(counts.values()))
    return torch.tensor(p_zero)




## Step 1.4 - Create a network

Now we're going to define a network, just like any other PyTorch network, only that this time, we will have only 1 layer, and it will be a quantum layer.

In [191]:
import torch.nn as nn

# ネットワークの作成

class Net(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__()
        self.qlayer = QLayer(
            quantum_program,
            execute,
            post_process,
            *args,
            **kwargs
        )
        self.qlayer.weight.data.uniform_(-0.1, 0.1)  # パラメータの初期化

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if x.ndimension() == 1:  # データが1次元の場合の処理
            input_0 = x.unsqueeze(0)
            input_1 = torch.zeros_like(input_0)
        elif x.size(1) == 1:  # データが (batch_size, 1) の形状の場合の処理
            input_0 = x
            input_1 = torch.zeros_like(x)
        else:  # データが (batch_size, 2) の形状の場合の処理
            input_0 = x[:, 0].unsqueeze(1)
            input_1 = x[:, 1].unsqueeze(1)

        inputs = torch.cat((input_0, input_1), dim=1)
        return self.qlayer(inputs)

model = Net()


# Step 2 - Choose a dataset, loss function, and optimizer

We will use the DATALOADER_NOT dataset, defined [here](https://docs.classiq.io/latest/reference-manual/built-in-algorithms/qml/qnn/datasets/) as well as [L1Loss](https://pytorch.org/docs/stable/generated/torch.nn.L1Loss.html) and [SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)

In [192]:
from classiq.applications.qnn.datasets import DATALOADER_NOT
import torch.nn as nn
import torch.optim as optim

        

# 損失関数とオプティマイザの定義
loss_func = nn.BCELoss()  # バイナリクロスエントロピー損失関数を使用
optimizer = optim.AdamW(model.parameters(), lr=0.001)  # AdamWオプティマイザを使用

# Step 3 - Train

For the training process, we will use a loop similar to [the one recommended by PyTorch](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#update-the-weights)

In [197]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


# 学習およびテストの準備
def train(model, data_loader, loss_func, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data, label in data_loader:
            optimizer.zero_grad()
            output = model(data)
            label = label.view(-1)  # ラベルの形状を修正
            loss = loss_func(output, label)
            loss.backward()

            # 勾配チェックを追加して問題を診断
            for name, param in model.named_parameters():
                if param.grad is not None:
                    print(f'Grad for {name}: {param.grad.norm().item()}')
                else:
                    print(f'No grad for {name}')
                if param.grad is not None and torch.all(param.grad == 0):
                    print(f'Gradient for {name} is zero.')

            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(data_loader)}')

    # 学習後のモデルを保存
    torch.save(model.state_dict(), './trained_model.pth')
    print("Trained model saved to trained_model.pth")
train(model, data_loader, loss_func, optimizer, epochs=20)

Grad for qlayer.weight: 0.0
Gradient for qlayer.weight is zero.
Epoch 1/20, Loss: 50.0
Grad for qlayer.weight: 0.0
Gradient for qlayer.weight is zero.
Epoch 2/20, Loss: 50.0
Grad for qlayer.weight: 0.0
Gradient for qlayer.weight is zero.
Epoch 3/20, Loss: 50.0
Grad for qlayer.weight: 0.0
Gradient for qlayer.weight is zero.
Epoch 4/20, Loss: 50.0
Grad for qlayer.weight: 0.0
Gradient for qlayer.weight is zero.
Epoch 5/20, Loss: 50.0
Grad for qlayer.weight: 0.0
Gradient for qlayer.weight is zero.
Epoch 6/20, Loss: 50.0
Grad for qlayer.weight: 0.0
Gradient for qlayer.weight is zero.
Epoch 7/20, Loss: 50.0
Grad for qlayer.weight: 0.0
Gradient for qlayer.weight is zero.
Epoch 8/20, Loss: 50.0
Grad for qlayer.weight: 0.0
Gradient for qlayer.weight is zero.
Epoch 9/20, Loss: 50.0
Grad for qlayer.weight: 0.0
Gradient for qlayer.weight is zero.
Epoch 10/20, Loss: 50.0
Grad for qlayer.weight: 0.0
Gradient for qlayer.weight is zero.
Epoch 11/20, Loss: 50.0
Grad for qlayer.weight: 0.0
Gradient for 

# Step 4 - Test

Lastly, we will test our network accuracy, using [the following answer](https://stackoverflow.com/questions/52176178/pytorch-model-accuracy-test#answer-64838681)

In [198]:
# テスト関数
def test(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, label in data_loader:
            output = model(data)
            label = label.view(-1)  # ラベルの形状を修正
            predicted = (output > 0.5).float()
            total += label.size(0)
            correct += (predicted == label).sum().item()
    print(f'Accuracy: {100 * correct / total}%')

test(model, DATALOADER_NOT)

# デバッグ情報を追加して再度確認
for data, label in DATALOADER_NOT:
    model = Net()
    output = model(data)
    label = label.view(-1)  # ラベルの形状を修正
    loss = loss_func(output, label)
    print(f"Data: {data}, Label: {label}, Output: {output}, Loss: {loss.item()}")
    break  # 最初のバッチのみを確認


Accuracy: 50.0%
Data: tensor([[3.1416],
        [0.0000]]), Label: tensor([1., 0.]), Output: tensor([0., 0.], grad_fn=<QLayerFunctionBackward>), Loss: 50.0


## reload model

In [199]:
model = Net()
model.load_state_dict(torch.load('./trained_model.pth'))
model.eval()
print("Model loaded successfully and set to evaluation mode.")

# テストの実行
test(model, DATALOADER_NOT)

Model loaded successfully and set to evaluation mode.
Accuracy: 50.0%
